# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# libraries for data
import pandas as pd
from sqlalchemy import create_engine

# libraries for NLP
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

#libraries for Modeling
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [56]:
# load data from database
engine = create_engine('sqlite:///DisasterTextTable.db')
df = pd.read_sql_table('DisasterTextTable',engine)

X = df['message'].values
Y = df.drop(['id','message','original','genre'],axis=1)

### 2. Write a tokenization function to process your text data

In [41]:
def tokenize(text):
    
    #text = re.sub(r"[^a-zA-Z0-9]", " ",text)
    
    tokens = word_tokenize(text.lower())
    lemmatizer = WordNetLemmatizer()
    stop_words = stopwords.words("english")
    
    # lemmatize and remove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [42]:
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()

#from sklearn.neighbors import KNeighborsClassifier
#clf = KNeighborsClassifier()

def model_pipeline():
    pipeline = Pipeline([
        ('vect',CountVectorizer(tokenizer=tokenize)),
        ('tfidf',TfidfTransformer()),
        ('multi',MultiOutputClassifier(clf))
    ])
    
    return pipeline

In [43]:
pipeline = model_pipeline()

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.3,random_state=0)

In [45]:
clf = pipeline.fit(X_train,y_train)
y_pred = clf.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [46]:
target_names = Y.columns

In [47]:
from sklearn.metrics import classification_report

print(classification_report(y_test,y_pred,target_names=target_names))

                        precision    recall  f1-score   support

               related       0.77      0.92      0.84      5947
               request       0.42      0.09      0.15      1316
                 offer       0.00      0.00      0.00        43
           aid_related       0.47      0.23      0.30      3268
          medical_help       0.11      0.00      0.01       635
      medical_products       0.13      0.01      0.01       421
     search_and_rescue       0.17      0.00      0.01       217
              security       0.00      0.00      0.00       146
              military       0.25      0.00      0.01       260
           child_alone       0.00      0.00      0.00         0
                 water       0.04      0.00      0.00       547
                  food       0.14      0.01      0.02       860
               shelter       0.16      0.01      0.02       710
              clothing       0.50      0.01      0.02       113
                 money       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [48]:
pipeline = model_pipeline()
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fb16f727e18>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('multi', MultiOutputClassifier(estimator=Pipeline(memory=None,
        steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip...oob_score=False, random_state=None, verbose=0,
   

In [55]:
from sklearn.model_selection import GridSearchCV

parameters = {
    'multi__estimator__multi__estimator__n_estimators' : list(range(10,600,100)),
    'multi__estimator__multi__estimator__max_depth': list(range(3,10,3)) 
}

cv = GridSearchCV(estimator = pipeline,
                 param_grid = parameters,
                 cv = 5)

cv.fit(X_train,y_train)

AttributeError: lower not found

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred = cv.predict(X_test)

In [ ]:
print(classification_report(y_test,y_pred,target_names=target_names))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [53]:
df.to_pickle("/disaster_models.pkl")

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.